In [4]:
%load_ext autoreload

In [5]:
%autoreload

In [169]:
for i in range(1):
    print(i)

0


In [9]:
import numpy as np
import pandas as pd
import warnings
import sys
from IPython.display import display, HTML
import tabulate #sudo pip3 install tabulate
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)
def update_column(ds,column):
    update_column=True
    if column in ds:
        if ds[column]:
            update_column=False
    return update_column

def _get_doi(surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        '''
        Search doi from http://search.crossref.org/ 
        '''
        import re
        import requests
        doi={}
        search=''
        if surname:
            search=surname
        if title:
            if len(search)>0:
                search=search+', '+title
        if other:
            if len(search)>0:
                search=search+', '+other
                
        r=requests.get('http://search.crossref.org/?q=%s' %search)
        urldoi='http://dx.doi.org/'
        doitmp=''
        if len(r.text.split(urldoi))>1:
            doitmp=r.text.split(urldoi)[1].split("\'>")[0].replace('&lt;','<').replace('&gt;','>')
        #check doi is right by searching for all words in doi -> output title
        if doitmp:
            json='https://api.crossref.org/v1/works/'
            rr=requests.get( json+urldoi+doitmp )
            if rr.status_code==200:
                if 'message' in rr.json():
                    chktitle = re.sub(r"\$.*?\$","",title) # better remove all math expressions
                    chktitle = re.sub(r"[^a-zA-Z0-9 ]", " ", chktitle).split(' ')
                    if chktitle:
                        if not -1 in [(rr.json()["message"]['title'][0]).find(w)  for w in chktitle]:
                            doi=rr.json()["message"]
                        
        return doi
class publications(object):
    '''Add Generic publication data'''
    pass
class articles(publications):
    '''Read csv file exported by Google Scholar Citations profile and
       automatically add informations about:
       DOI: from title and author
       Journal title: from DOI
       ISSN of Jornal
       Impact factor of journal
       Institution_Authors: requires Data Base
       '''
    journal=pd.Series()
    columns=pd.Series({'Full_Name':'Full_Name','Author_Names':'Author_Names','Control':'Control',\
                      'Institution_Authors':'Institution_Authors','Institution_Group':'Institution_Group'})

    def __init__(self,csv_file='citations.csv',citations_file=None,authors_file=None,group_file=None):
        #DEBUG: check file
        self.articles=pd.read_csv(csv_file).fillna('')
        #Fix problem with column Authors
        if self.articles.shape[0]>0 and self.articles.columns[0].find('Authors')>-1:
            self.articles=self.articles.rename(columns={self.articles.columns[0]:'Authors'})
        if citations_file:
            #DEBUG: check file
            self.citations=pd.read_csv(citations_file).fillna('')
            
        if authors_file:
            #DEBUG: chek is file exists,
            self.institution_authors=pd.read_json(authors_file).fillna('')
            
        if group_file:
            #DEBUG: chek is file exists,
            self.institution_group=pd.read_json(group_file).fillna('')
            
    def add_institution_author(self):
        #DEBUG: Check and load file
        full_name=input('Full name: ')
        #DEBUG: Ask for more author names and update author names separated by semicolons
        author_names=input('Author names\n(Example: Juan Perez; J. Perez; J. Pérez):\n')
        #DEBUG: default to 0
        control=input('Additional identitication number: ') 
        
        #DEBUG: if not file creates it
        iffile=False
        if not iffile:
            self.institution_authors=pd.DataFrame()
            
        self.institution_authors=self.institution_authors.append({self.columns.Full_Name:full_name,\
                                                              self.columns.Author_Names:author_names,\
                                                              self.columns.Control:control},ignore_index=True)
        self.institution_authors.to_json('authors.json')
        
    def add_institution_group(self):
        wrn='Author not found'
        search_query=input('Find author by first name and first surname\n(Example: Juan Pérez);\n')
        sql=search_query.split(' ')
        #DEBUG: low_case
        if len(sql)>0:
            sql_name=sql[0]
            if len(sql)>1:
                sql_surname=sql[1]
            else:
                sql_surname=''
            
            author_match=self.institution_authors[\
                             np.logical_and( self.institution_authors.Full_Name.str.contains(sql_name),\
                                             self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                author_match=self.institution_authors[\
                             np.logical_or( self.institution_authors.Full_Name.str.contains(sql_name),\
                                            self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                warnings.warn(wrn)
            else:    
                author_match=author_match.reset_index(drop=True)
                author_match['Number']=author_match.index
                if author_match.shape[0]==1:
                    author_Series=author_match.ix[0]
                    #print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                else:
                    print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                    ai=input('Give Number of line:')
                    author_Series=author_match.ix[ai]
                Group=input('Group name for\n%s\n(Example: IA: Inteligencia Artificial)' %author_Series.Full_Name)
                #DEBUG: if not file creates it
                iffile=False
                if not iffile:
                    self.institution_group=pd.DataFrame()  
                    self.institution_group=self.institution_group.append({self.columns.Full_Name:author_Series.Full_Name,\
                                                                  self.columns.Institution_Group:Group},ignore_index=True)
                    self.institution_group.to_json('groups.json')
        else:
            warnings.warn(wrn)

        
        
    def get_doi(self,surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        return _get_doi(surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other='')
    
    def articles_update(self):
        fulldoi=pd.DataFrame()
        newcolumns=['Institution_Authors','DOI']
        for newcolumn in newcolumns:
            if not newcolumn in self.articles.columns:
                self.articles[newcolumn]=''

        self.articles=self.articles.reset_index(drop=True)
        print('Updating entry:',end="")
        for i in range(self.articles.shape[0]):
            print('%d.' %i,end="")
            #Update Full names:
            if self.articles.ix[i].Authors:
                inst_auth=''
                inst_auth_sep=';'
                for a in filter(None, self.articles.ix[0].Authors.replace('; ',';').split(';') ):
                    af=self.institution_authors[self.institution_authors.Author_Names.str.contains(a)].reset_index(drop=True)
                    if af.shape[0]==1:
                        if inst_auth:
                            inst_auth=inst_auth+inst_auth_sep
                        if af.Full_Name.values.shape[0]==1:
                            inst_auth=inst_auth+af.Full_Name.values[0]
                    elif af.shape[0]>1:
                        print('Improve real name matching')
                    if update_column(self.articles.ix[i],'Institution_Authors'):
                        self.articles.loc[i,'Institution_Authors']=inst_auth
            #Update DOI:
            rr=_get_doi(surname=self.articles.ix[i].Authors.split(';')[0].split(',')[0],title=self.articles.ix[i].Title)
            if len(rr)>0:
                if update_column(self.articles.ix[i],'DOI') and 'URL' in rr:
                        self.articles.loc[i,'DOI']=rr['URL']
                        
            fulldoi=fulldoi.append(rr,ignore_index=True)
            
        return fulldoi.fillna('')
                        
            

Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [10]:
arts=articles('citations.csv',authors_file='authors.json',group_file='groups.json')
arts.institution_authors

,Author_Names,Control,Full_Name
0,"Zapata, Oscar;Zapata, O.",0,Óscar Alberto Zapata Noreña


In [11]:
arts.institution_group

,Full_Name,Institution_Group
0,Óscar Alberto Zapata Noreña,GFIF


In [12]:
fulldoi=arts.articles_update()

Updating entry:0.1.2.3.4.5.6.7.8.9.10.11.12.

In [13]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,DOI
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.79.013011
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.85.023523
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1088/1475-7516/2010/10/033
3,"Sierra, D Aristizabal; Restrepo, Diego; Zapata, Oscar;",Decaying neutralino dark matter in anomalous U (1) H models,Physical Review D,80,5,55010,2009,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.80.055010
4,"Klasen, Michael; Yaguna, Carlos E; Ruiz-Álvarez, José D; Restrepo, Diego; Zapata, Oscar;",Scalar dark matter and fermion coannihilations in the radiative seesaw model,Journal of Cosmology and Astroparticle Physics,2013,4,44,2013,IOP Publishing,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1088/1475-7516/2013/04/044
5,"Florez, Andres; Restrepo, Diego; Velasquez, Mauricio; Zapata, Oscar;",Baryonic violation of R parity from anomalous U (1) H,Physical Review D,87,9,95010,2013,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.87.095010
6,"Molinaro, Emiliano; Yaguna, Carlos E; Zapata, Oscar;",FIMP realization of the scotogenic model,Journal of Cosmology and Astroparticle Physics,2014,7,15,2014,IOP Publishing,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1088/1475-7516/2014/07/015
7,"Restrepo, Diego; Rivera, Andrés; Sánchez-Peláez, Marta; Zapata, Oscar; Tangarife, Walter;",Radiative neutrino masses in the singlet-doublet fermion dark matter model with scalar singlets,Physical Review D,92,1,13005,2015,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.92.013005
8,"Arbeláez, Carolina; Longas, Robinson; Restrepo, Diego; Zapata, Oscar;",Fermion dark matter from SO (10) GUTs,Physical Review D,93,1,13012,2016,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.93.013012
9,"Ponce, William A; Zapata, Oscar;",Lepton masses and mixing without Yukawa hierarchies,Physical Review D,74,9,93007,2006,APS,Óscar Alberto Zapata Noreña,http://dx.doi.org/10.1103/physrevd.74.093007


In [17]:
fulldoi=fulldoi.fillna('')

In [25]:
rr=fulldoi.ix[2]
rr

DOI                                                                                                                                                                                           10.1088/1475-7516/2010/10/033
ISSN                                                                                                                                                                                                            [1475-7516]
URL                                                                                                                                                                         http://dx.doi.org/10.1088/1475-7516/2010/10/033
article-number                                                                                                                                                                                                             
author              [{'affiliation': [], 'given': 'Ki-Young', 'family': 'Choi'}, {'affiliation': [], 'given': 'Diego', '

In [22]:
print(rr.ISSN,rr['article-number'],rr.publisher,rr.title,rr.volume,rr.page,rr.funder)


['1550-7998', '1550-2368'] 013011 American Physical Society (APS) ['Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals'] 79  


In [26]:
cc=pd.read_csv('citations.csv')

In [27]:
cc[:3].to_csv('citations.csv')

,﻿Authors,Title,Publication,Volume,Number,Pages,Year,Publisher
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing
